# Create a dataframe per domain
each dataframe has the source code and refactoration per line

# Split data into domain

In [ ]:
import pandas as pd

df_yes_method = pd.read_csv('../data/root/output/commons-csv/yes-method.csv')
df_targets = pd.read_csv('../data/root/output/commons-csv/yes.csv')
df_yes_method["refactoring"] = None


domains = ['fdroid', 'github', 'apache']
datasets = {}
for domain in domains:
    print("Spliting {}".format(domain))
    new_df = df_yes_method[df_yes_method['dataset'] == domain]    
    new_df.to_pickle('dataframe_{}.pickle'.format(domain))

# Open each spirce code an search for refactorations

In [ ]:
import pandas as pd
import numpy as np
import nltk
import json

df_yes_method = pd.read_csv('all-yes-method-2.csv')
df_targets = pd.read_csv('all-yes-2.csv')

df_yes_method["refactoring"] = None

In [ ]:
import pandas as pd
import numpy as np
import nltk
import json
from multiprocessing import Pool

df_yes_method = pd.read_csv('all-yes-method-2.csv')
df_targets = pd.read_csv('all-yes-2.csv')
df_yes_method["refactoring"] = None

# df_yes_method = pd.read_csv('../data/root/output/commons-csv/yes-method.csv')
# df_targets = pd.read_csv('../data/root/output/commons-csv/yes.csv')
# df_yes_method["refactoring"] = None

def get_refactoring_row(row):
    return df_targets[(df_targets.dataset == row["dataset"]) & (df_targets.project == row["project"]) &\
                      (df_targets.parentCommit == row["refactorCommit"]) & (df_targets.method == row["method"])]

def tokenize_line(line):
    return ' '.join(nltk.tokenize.wordpunct_tokenize(line))

def get_file_lines(row):
    domain = row["dataset"]
    with open('{}/root/output/{}/storage/{}/before-refactoring/{}'.format(domain,
                                                                          row["project"].replace(".git", ""),
                                                                          row["refactorCommit"],
                                                                          row["path"]), 'r') as text_file:
        lines = text_file.readlines()
        lines = [tokenize_line(line) for line in lines]
        return lines

def label_file(row, label, trainset):
    hash_label = "{}-{}-{}".format(row["project"], row["refactorCommit"], row["path"])
    if hash_label not in trainset:
        try: # try to get the file content
            lines = get_file_lines(row)
            labels = ['no' for line in lines]
        except: # if some error occurr, just return the original data
            return trainset
    else:
        lines = trainset[hash_label]['lines']
        labels = trainset[hash_label]['labels']
    try:
        labels[row['line'] - 1] = label
    except:
        print(labels)
    trainset[hash_label] = {'lines': lines, 'labels': labels}  
    
    return trainset

def prepare_files(domain):
    print("Preparing {}".format(domain))
    trainset = {} # hash_label = project-parentCommit-path
    i = 0
    df_yes = pd.read_pickle('dataframe_{}.pickle'.format(domain))
    for index, row in df_yes.iterrows():
        i += 1
        if 'Test.java' not in row["path"]:
            refactoring_row = get_refactoring_row(row)
            if len(refactoring_row["refactoring"]) > 0:
                label = list(refactoring_row["refactoring"])[0]
                trainset = label_file(row, label.replace(" ","_"), trainset)
        if i % 1000 == 0:
            print("{} of {} rows processed".format(i, len(df_yes)))
    
    with open('trainset_{}.json'.format(domain), 'w') as json_file:
        json_file.write(json.dumps(trainset))

domains = ['fdroid', 'github', 'apache']
pool = Pool(8)
pool.map(prepare_files, domains)